In [2]:
import numpy as np
import torch
from numba import jit
import sys
sys.path.append('/home/ahabis/1-Click_project')
from config import *
import time

In [3]:
u = np.load('pairwise_distance'+ str(parameters['n_embedding'])+'.npy')